In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import dash_bio

In [3]:
#create pandas dataframe
path = "tidy.csv"
pathMet = "metadata.csv"
tidy = pd.read_csv(path, sep=",")
tidyMet = pd.read_csv(pathMet, sep=";", index_col=0)

#remove samples which are not in the metadata index column (quality controle etc)
tidy = tidy[ (tidy["R.FileName"].isin(tidyMet.index)) ]
tidyMer    = pd.merge(tidy, tidyMet, how="left", on="R.FileName")
tidySub = tidyMer[["R.FileName", "uniprot", "meanAbu", "Cancer"]]
tidySub.Cancer.value_counts()

Breast Cancer        84630
Lung Cancer          81900
Pancreatic Cancer    81900
Colorectal Cancer    81900
Healthy              81900
Prostate Cancer      81900
Name: Cancer, dtype: int64

In [4]:
tidySub.head()

,R.FileName,uniprot,meanAbu,Cancer
0,J_D201211_MDIA_P705_SExp01-BGSID-1_R01,A0A075B6H9,11601.398438,Lung Cancer
1,J_D201211_MDIA_P705_SExp01-BGSID-1_R01,A0A075B6I0,3827.466797,Lung Cancer
2,J_D201211_MDIA_P705_SExp01-BGSID-1_R01,A0A075B6I1,114115.796875,Lung Cancer
3,J_D201211_MDIA_P705_SExp01-BGSID-1_R01,A0A075B6I7,4293.524902,Lung Cancer
4,J_D201211_MDIA_P705_SExp01-BGSID-1_R01,A0A075B6J1,5969.785645,Lung Cancer


In [5]:
tidySub[(tidySub.uniprot == 'A0A075B6H9')&(tidySub.Cancer == 'Healthy')].head()

,R.FileName,uniprot,meanAbu,Cancer
19110,J_D201211_MDIA_P705_SExp01-BGSID-105_R01,A0A075B6H9,168020.156250,Healthy
30030,J_D201211_MDIA_P705_SExp01-BGSID-109_R01,A0A075B6H9,9181.585938,Healthy
46410,J_D201211_MDIA_P705_SExp01-BGSID-114_R01,A0A075B6H9,5460.259277,Healthy
60060,J_D201211_MDIA_P705_SExp01-BGSID-119_R01,A0A075B6H9,36868.132812,Healthy
65520,J_D201211_MDIA_P705_SExp01-BGSID-13_R01,A0A075B6H9,4813.011719,Healthy


For the pvalues, we need data stored in a list for each protein and cancertype 

In [6]:
ValueDF = tidySub.groupby(['uniprot','Cancer'])['meanAbu'].apply(list)

In [7]:
ValueDF = ValueDF.reset_index()

In [8]:
ValueDF[(ValueDF.uniprot == 'A0A075B6H9') & (ValueDF.Cancer == 'Healthy')].meanAbu

2    [168020.15625, 9181.5859375, 5460.25927734375,...
Name: meanAbu, dtype: object

In [9]:
ValueDF

,uniprot,Cancer,meanAbu
0,A0A075B6H9,Breast Cancer,"[35277.67578125, 34095.4765625, 3512.60546875,..."
1,A0A075B6H9,Colorectal Cancer,"[38555.5703125, 37487.71875, 28599.8984375, 42..."
2,A0A075B6H9,Healthy,"[168020.15625, 9181.5859375, 5460.25927734375,..."
3,A0A075B6H9,Lung Cancer,"[11601.3984375, 4950.046875, 4343.33154296875,..."
4,A0A075B6H9,Pancreatic Cancer,"[14946.3359375, 4416.61767578125, 30662.833984..."
...,...,...,...
16375,Q9Y6Z7,Colorectal Cancer,"[125766.78125, 101316.0546875, 92774.53125, 81..."
16376,Q9Y6Z7,Healthy,"[218686.546875, 66921.515625, 133171.328125, 7..."
16377,Q9Y6Z7,Lung Cancer,"[256844.078125, 99598.9375, 163073.59375, 1547..."
16378,Q9Y6Z7,Pancreatic Cancer,"[186635.421875, 242902.84375, 137199.046875, 5..."


In [10]:
from scipy import stats
x = ValueDF.iloc[2].meanAbu
y = ValueDF.iloc[0].meanAbu
f = stats.ttest_ind(x, y)

For the log2 values, we use MeanDF

In [11]:
MeanDF = tidySub.groupby(['uniprot','Cancer'])['meanAbu'].mean()

In [12]:
MeanDF = pd.DataFrame(MeanDF)
MeanDF = MeanDF.reset_index()
MeanDF['uniprot']

0        A0A075B6H9
1        A0A075B6H9
2        A0A075B6H9
3        A0A075B6H9
4        A0A075B6H9
            ...    
16375        Q9Y6Z7
16376        Q9Y6Z7
16377        Q9Y6Z7
16378        Q9Y6Z7
16379        Q9Y6Z7
Name: uniprot, Length: 16380, dtype: object

In [13]:
MeanDF[MeanDF.uniprot == 'A0A075B6H9']

,uniprot,Cancer,meanAbu
0,A0A075B6H9,Breast Cancer,13187.140263
1,A0A075B6H9,Colorectal Cancer,11832.107357
2,A0A075B6H9,Healthy,39325.718099
3,A0A075B6H9,Lung Cancer,19166.234025
4,A0A075B6H9,Pancreatic Cancer,30563.667741
5,A0A075B6H9,Prostate Cancer,12818.901481


In [14]:
pivMeanDF = MeanDF.pivot_table(values='meanAbu', index=['uniprot'], columns='Cancer')

In [15]:
MeanDF = MeanDF.reset_index()

In [16]:
pivMeanDF = pivMeanDF.reset_index()

In [17]:
breastcancer = pd.DataFrame(pivMeanDF[['uniprot', 'Breast Cancer', 'Healthy']]
                            )

In [18]:
ValueDF[(ValueDF.uniprot == 'A0A075B6H9') & (ValueDF.Cancer == 'Breast Cancer')]
ValueDF[(ValueDF.uniprot == 'A0A075B6H9') & (ValueDF.Cancer == 'Healthy')]

,uniprot,Cancer,meanAbu
2,A0A075B6H9,Healthy,"[168020.15625, 9181.5859375, 5460.25927734375,..."


In [19]:
len(x[:30])

30

In [20]:
stats.ttest_ind(x[:30], y).pvalue

0.021226129911409028

In [21]:
breastcancer = breastcancer.set_index('uniprot')

In [22]:
masks = (breastcancer['uniprot'] == 'A0A075B6H9')
breastcancer.loc[masks].pvalue.mask(masks, 1)
print(breastcancer.loc[masks].pvalue.values)

KeyError: 'uniprot'

In [389]:
breastcancer.at['A0A075B6H9', 'pvalue'] = 'False'

In [393]:
from scipy import stats

x = ValueDF[(ValueDF.uniprot == protein) & (ValueDF.Cancer == 'Breast Cancer')].meanAbu.values[0]
y = ValueDF[(ValueDF.uniprot == protein) & (ValueDF.Cancer == 'Healthy')].meanAbu.values[0]
print(y)
breastcancer.at[protein, 'pvalue']  = stats.ttest_ind(x, y).pvalue

[168020.15625, 9181.5859375, 5460.25927734375, 36868.1328125, 4813.01171875, 3942.99755859375, 3867.3271484375, 6215.5244140625, 5632.58642578125, 4430.26953125, 4153.984375, 18559.1953125, 4144.66064453125, 9355.5283203125, 5622.3095703125, 5443.46875, 21316.87109375, 38519.265625, 16850.087890625, 102725.84375, 8596.5517578125, 2620.28100585938, 4085.49926757813, 54102.7109375, 63219.26171875, 89984.1875, 54763.09765625, 273791.625, 42764.18359375, 110721.078125]


In [26]:
for protein in breastcancer.index:
    x = ValueDF[(ValueDF.uniprot == protein) & (ValueDF.Cancer == 'Breast Cancer')].meanAbu.values[0]
    y = ValueDF[(ValueDF.uniprot == protein) & (ValueDF.Cancer == 'Healthy')].meanAbu.values[0]
    breastcancer.at[protein, 'pvalue']  = stats.ttest_ind(x, y).pvalue

In [440]:
#type(stats.ttest_ind(x, y).pvalue)
type(breastcancer.at['A0A075B6H9', 'pvalue'])

numpy.float64

In [27]:
breastcancer.head()

Cancer,Breast Cancer,Healthy,L2F,pvalue
uniprot,,,,
A0A075B6H9,13187.140263,39325.718099,1.115172,0.021226
A0A075B6I0,36317.173757,176242.021566,1.150434,0.004225
A0A075B6I1,7822.352153,14905.960042,1.071924,0.054360
A0A075B6I7,4667.722373,7371.170549,1.054082,0.248206
A0A075B6J1,4698.463450,11839.818555,1.109312,0.037755


In [32]:
from math import log2
breastcancer['L2F'] = (breastcancer['Breast Cancer'].apply(log2) / breastcancer['Healthy'].apply(log2))

In [34]:
breastcancer.L2F

0       0.896723
1       0.869237
2       0.932902
3       0.948693
4       0.901460
          ...   
2725    0.989437
2726    0.971692
2727    1.016711
2728    1.000083
2729    1.005832
Name: L2F, Length: 2730, dtype: float64

In [108]:
# test if we only have two rows per protein
bdata[bdata['variable']=='A0A075B6H9']

,HasCancer,variable,value
0,False,A0A075B6H9,39325.718099
2730,True,A0A075B6H9,17484.958055


In [113]:
bdata.head()

Cancer         HasCancer  variable  
Breast Cancer  True       A0A075B6H9    13187.140263
                          A0A075B6I0    36317.173757
                          A0A075B6I1     7822.352153
                          A0A075B6I7     4667.722373
                          A0A075B6J1     4698.463450
Name: value, dtype: float64

In [110]:
bdata = data.melt(id_vars='Cancer')
bdata['HasCancer'] = bdata['Cancer']!='Healthy'

In [112]:
bdata = bdata.groupby(['Cancer','HasCancer', 'variable'])['value'].mean()

In [116]:
bdata=bdata.reset_index()
bdata = bdata.groupby(['HasCancer', 'variable'])['value'].apply(list)

In [122]:
bdata = bdata.reset_index()

In [124]:
bdata[bdata.variable=='A0A075B6H9']

,HasCancer,variable,value
0,False,A0A075B6H9,[39325.718098958336]
2730,True,A0A075B6H9,"[13187.140262726814, 11832.107356770834, 19166..."


In [53]:
del bdata['Cancer']

In [126]:
bdata = bdata[['variable', 'HasCancer', 'value']]

In [128]:
bdata[bdata.variable=='A0A075B6I1']

,variable,HasCancer,value
2,A0A075B6I1,False,[14905.960042317709]
2732,A0A075B6I1,True,"[7822.352153162802, 9022.135489908855, 11532.6..."


In [ ]:
bdata.pivot_table(index= 'variable',
                   columns='HasCancer',
                   values='value')

In [24]:
from math import log10
breastcancer.pvalue.apply((log10))
breastcancer.pvalue.multiply(-1)

AttributeError: 'DataFrame' object has no attribute 'pvalue'

In [444]:
breastcancer.pvalue.values

array([0.021226129911409028, 0.004225028876737513, 0.05435961919879539,
       ..., 0.11821295573629942, 0.9954835192088747, 0.7090635563859037],
      dtype=object)

In [432]:
breastcancer.head()
type(breastcancer.pvalue[[1]])
breastcancer.pvalue[[1]]

uniprot
A0A075B6I0    0.004225
Name: pvalue, dtype: object

In [30]:
breastcancer = breastcancer.reset_index()

In [467]:
import joblib
joblib.dump(breastcancer, 'breastcancerdataframe')

['breastcancerdataframe']

In [43]:
dash_bio.VolcanoPlot(
    dataframe=breastcancer,
    effect_size='L2F',
    p='pvalue',
    snp = 'uniprot',
    gene=None,
    logp=True
)

In [454]:
breastcancer = breastcancer.astype({"pvalue": float}, errors='raise') 

In [456]:
breastcancer.dtypes

Cancer
Breast Cancer    float64
Healthy          float64
L2F              float64
pvalue           float64
dtype: object

In [158]:
data[data.index == 'J_D201211_MDIA_P705_SExp01-BGSID-100_R01']

,A0A075B6H9,A0A075B6I0,A0A075B6I1,A0A075B6I7,A0A075B6J1,A0A075B6J9,A0A075B6K4,A0A075B6K5,A0A075B6P5;P01615,A0A075B6R2,...,Q9Y6C2,Q9Y6E0,Q9Y6E2,Q9Y6G9,Q9Y6L6,Q9Y6N7,Q9Y6R7,Q9Y6Y9,Q9Y6Z7,Cancer
R.FileName,,,,,,,,,,,,,,,,,,,,,
J_D201211_MDIA_P705_SExp01-BGSID-100_R01,14946.335938,44693.898438,5286.701172,2721.810303,6439.688965,4968.654297,9251.992188,40399.160156,76319.507812,50205.507812,...,46711.242188,4999.735352,3843.583496,3900.420654,4008.093262,30271.335938,5562199.5,20829.845703,186635.421875,Pancreatic Cancer
